<a href="https://colab.research.google.com/github/parmarsuraj99/10DaysofMLChallenge/blob/master/day6/Day_6_10daysofMLChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [0]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [0]:
!mkdir data
!unzip /content/chest-xray-pneumonia.zip -d data/;
!rm -r /content/data/chest_xray/chest_xray

In [0]:
import os
import gc
import glob
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt

In [0]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, load_img

from tensorflow.keras.layers import Dense, Dropout,  Conv2D, Input, Lambda, Flatten, TimeDistributed
from tensorflow.keras.layers import Add, Reshape, MaxPooling2D, Concatenate, Embedding, RepeatVector, BatchNormalization
from tensorflow.keras import backend as K

from keras.utils import np_utils
from keras.engine.topology import Layer
from tensorflow.keras.callbacks import TensorBoard

In [0]:
normal_files = glob.glob("/content/data/chest_xray/train/NORMAL/*.jpeg")
pnm_files = glob.glob("/content/data/chest_xray/train/PNEUMONIA/*.jpeg")
random = np.random.randint(0, len(normal_files), 5)
print("NORMAL shapes")
for normal in random:
    img = load_img(normal_files[normal])
    print(img_to_array(img).shape)

random = np.random.randint(0, len(pnm_files), 5)
print("PNEUMONIA shapes")
for i in random:
    img = load_img(pnm_files[i])
    print(img_to_array(img).shape)

In [0]:
#https://github.com/mahimanzum/visual_attention_imeplemented


def MultiHeadsAttModel(l=8*8, d=512, dv=64, dout=512, nv = 8 ):

    v1 = Input(shape = (l, d))
    q1 = Input(shape = (l, d))
    k1 = Input(shape = (l, d))

    v2 = Dense(dv*nv, activation = "relu")(v1)
    q2 = Dense(dv*nv, activation = "relu")(q1)
    k2 = Dense(dv*nv, activation = "relu")(k1)

    v = Reshape([l, nv, dv])(v2)
    q = Reshape([l, nv, dv])(q2)
    k = Reshape([l, nv, dv])(k2)
    att = tf.einsum('baik,baij->bakj',q, k)/np.sqrt(dv)
    #att = Lambda(lambda x: K.batch_dot(x[0],x[1] ,axes=[-1,-1]) / np.sqrt(dv),output_shape=(l, nv, nv))([q,k])# l, nv, nv
    #att = tf.einsum('', q, k)
    att = Lambda(lambda x:  K.softmax(x) , output_shape=(l, nv, nv))(att)
    out = tf.einsum('bajk,baik->baji',att, v)
    #out = Lambda(lambda x: K.batch_dot(x[0], x[1],axes=[2,2]),  output_shape=(l, nv, dv))([att, v])
    out = Reshape([l, d])(out)
    
    out = Add()([out, q1])

    out = Dense(dout, activation = "relu")(out)

    return  Model(inputs=[q1,k1,v1], outputs=out)

class NormL(Layer):

    def __init__(self, **kwargs):
        super(NormL, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.a = self.add_weight(name='kernel', 
                                      shape=(1,input_shape[-1]),
                                      initializer='ones',
                                      trainable=True)
        self.b = self.add_weight(name='kernel', 
                                      shape=(1,input_shape[-1]),
                                      initializer='zeros',
                                      trainable=True)
        super(NormL, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        eps = 0.000001
        mu = K.mean(x, keepdims=True, axis=-1)
        sigma = K.std(x, keepdims=True, axis=-1)
        ln_out = (x - mu) / (sigma + eps)
        return ln_out*self.a + self.b

    def compute_output_shape(self, input_shape):
        return input_shape

In [0]:
batch_size = 128
input_shape = (224, 224)

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [0]:
train_generator = train_datagen.flow_from_directory(
        '/content/data/chest_xray/train',
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='binary')
validation_generator = valid_datagen.flow_from_directory(
        '/content/data/chest_xray/val',
        target_size=input_shape,
        batch_size=16,
        class_mode='binary')


In [0]:
train_steps_per_epoch = train_generator.n//train_generator.batch_size
valid_steps_per_epoch = validation_generator.n//validation_generator.batch_size

In [0]:
gc.collect()

In [0]:
attn = True

In [0]:
inp = Input(shape = (224, 224, 3))
x = Conv2D(32,(2,2),activation='relu', padding='same')(inp)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Conv2D(64,(2,2),activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = Conv2D(64,(2,2),activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2), padding='same')(x)

if True:
    x = Reshape([28*28,64])(x)    
    att = MultiHeadsAttModel(l=28*28, d=64 , dv=8, dout=32, nv = 8 )
    x = att([x,x,x])
    x = Reshape([28,28,32])(x)   
    #x = NormL()(x)
    x = BatchNormalization()(x)

x = Flatten()(x) 
x = Dense(256, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inp, outputs=x)

In [0]:
model.summary()

In [0]:
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(input_shape[0], input_shape[1], 3), activation='sigmoid'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(layers.Conv2D(32, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.summary()

In [0]:
gc.collect()

In [0]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.0001)
optim = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.binary_crossentropy,
              metrics=["accuracy"],
              
              )

In [0]:
history1 = model.fit(train_generator,
                    steps_per_epoch = train_steps_per_epoch,
                    epochs = 10,
                    validation_data = validation_generator,
                    validation_steps = valid_steps_per_epoch,
                    callbacks=[reduce_lr],
                    )

In [0]:
history2 = model.fit(train_generator,
                    steps_per_epoch = train_steps_per_epoch,
                    epochs = 10,
                    validation_data = validation_generator,
                    validation_steps = valid_steps_per_epoch,
                    callbacks=[reduce_lr],
                    )

In [0]:
history = history2

In [0]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()